In [1]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pickle

data = pd.read_csv('metadata.csv', low_memory=False)
f=data['genres']
data = data.dropna(axis=0, subset=['genres'])

lst=list()
data['genres'] = data['genres'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
data['new']=data['genres']

def eng(x):
    llsst=[]
    for i in x:
        for j in i:
            if j=='name':
                llsst.append(i[j])
    return llsst


data['new']=data['genres'].apply(lambda x: eng(x))
data['full_text'] = data['original_title'] + "_" + data['overview']
newdata=pd.DataFrame()
newdata['full_text']=data['full_text']
newdata['new']=data['new']

newdata['lenth']=newdata['new'].str.len()

import pandas as pd

df = pd.DataFrame(newdata)

dummies = df['new'].apply(lambda x: pd.Series([1] * len(x), index=x)).fillna(0)

# Concatenate the dummy variables with the original DataFrame
df_with_dummies = pd.concat([df, dummies], axis=1)

# Drop the original 'lists' column if needed
df_with_dummies = df_with_dummies.drop(columns=['new'])

df_with_dummies['new']=data['new']

df_with_dummies.dropna(subset = ['full_text'], inplace=True)
from sklearn.model_selection import train_test_split
#df_with_dummies=df_with_dummies[:200]
x = df_with_dummies['full_text']
y=df_with_dummies['new']


In [2]:
#model initial and parameter fixing
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
mlb = MultiLabelBinarizer()
rf_classifier = RandomForestClassifier(n_estimators=5000)

X = tfidf_vectorizer.fit_transform(x).toarray()
Y = mlb.fit_transform(y)
shape=list(X.shape)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)
with open('mlb.pkl', 'wb') as mlb_file:
    pickle.dump(mlb, mlb_file)

In [3]:
j=200
i=0
while j<shape[0]:
    text_data = X[i:j]
    labels = Y[i:j]
    rf_classifier.fit(text_data, labels)
    i=j
    j+=200
    print(j)

400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
18000
18200
18400
18600
18800
19000
19200
19400
19600
19800
20000
20200
20400
20600
20800
21000
21200
21400
21600
21800
22000
22200
22400
22600
22800
23000
23200
23400
23600
23800
24000
24200
24400
24600
24800
25000
25200
25400
25600
25800
26000
26200
26400
26600
26800
27000
27200
27400
27600
27800
28000
28200
28400
28600
28800
29000
29200
29400
29600
29800
30000
30200
30400
30600
30800
31000
31200
31400
31600
31800
32000
32200
32400
32600
32800
33000
33200
33400
33600
33800
34000
34200
34400
34600
34800
35000
35200
3

In [5]:
text_data = X[i:shape[0]]
labels = Y[i:shape[0]]
rf_classifier.fit(text_data, labels)

RandomForestClassifier(n_estimators=5000)

In [6]:
with open('random_forest_classifier.pkl', 'wb') as classifier_file:
    pickle.dump(rf_classifier, classifier_file)

In [98]:
input_text =["Toy Story_Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."]

input_vector = tfidf_vectorizer.transform(input_text).toarray()

In [99]:
input_vector.shape

(1, 5000)

In [100]:
predicted_probabilities = rf_classifier.predict_proba(input_vector)

In [101]:
predicted_probabilities

[array([[0.9732, 0.0268]]),
 array([[0.9812, 0.0188]]),
 array([[0.95737881, 0.04262119]]),
 array([[1.]]),
 array([[1.]]),
 array([[1.]]),
 array([[0.8384, 0.1616]]),
 array([[9.996e-01, 4.000e-04]]),
 array([[0.9534, 0.0466]]),
 array([[0.4412, 0.5588]]),
 array([[0.9174, 0.0826]]),
 array([[0.9574, 0.0426]]),
 array([[0.9514, 0.0486]]),
 array([[1.]]),
 array([[9.998e-01, 2.000e-04]]),
 array([[0.9968, 0.0032]]),
 array([[1.]]),
 array([[0.9328, 0.0672]]),
 array([[0.9968, 0.0032]]),
 array([[1.]]),
 array([[1.]]),
 array([[1.]]),
 array([[0.444, 0.556]]),
 array([[0.9954, 0.0046]]),
 array([[1.]]),
 array([[0.9954, 0.0046]]),
 array([[1.]]),
 array([[1.]]),
 array([[0.9888, 0.0112]]),
 array([[1.]]),
 array([[1.]]),
 array([[1.]])]

In [60]:
import numpy as np

# Iterate through predicted_probabilities and make sure each array has two elements
for i in range(len(predicted_probabilities)):
    # Check if the array has only one element
    if len(predicted_probabilities[i][0]) == 1:
        # If the array has only one element, assume the second element is the complement
        complement = 1 - predicted_probabilities[i][0][0]
        predicted_probabilities[i][0] = np.array([predicted_probabilities[i][0][0], complement])

# Now all arrays in predicted_probabilities should have two elements
print(predicted_probabilities)



ValueError: could not broadcast input array from shape (2,) into shape (1,)

In [83]:
new=list()
for i in range(len(predicted_probabilities)):
    
    new.append(1-predicted_probabilities[i][0][0])
treshold=.3
pred=list()
for i in new:
    if i >= treshold:
        pred.append(1)
    else:
        pred.append(0)


In [84]:
l=np.array([pred])
predicted_labels_text = mlb.inverse_transform(l)

In [85]:
predicted_labels_text

[('Drama', 'Romance')]

In [96]:
data.to_csv('out.csv')  

In [102]:
print(data['full_text'][0])
print(type(data['new'][0]))

Toy Story_Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.
<class 'list'>


In [105]:
data['new'][0][0]

'Animation'